In [4]:
import odrive
from odrive.enums import *
import time
import math
import numpy as np
import sympy as sp
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt
plt.ion()
import matplotlib as mpl
mpl.use('Qt5Agg')
from scipy.optimize import minimize

# Forward Kinematics & IK & Jacobian (Joint space -- Foot space)

In [5]:
(thetaL_sym, thetaR_sym) = sp.symbols("""thetaL_sym thetaR_sym """, real = True)

def T(theta, x, y):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x], 
                      [sp.sin(theta), sp.cos(theta), y],
                      [0, 0, 1]])

l1 = 0.09
l2 = 0.16
w  = 0.07

x_r = -w/2 + l1*sp.cos(thetaR_sym)
x_l = w/2 + l1*sp.cos(thetaL_sym)

y_r = l1*sp.sin(thetaR_sym)
y_l = l1*sp.sin(thetaL_sym)

theta3_sym = sp.atan2(y_l - y_r, x_l - x_r)
L = sp.sqrt((x_l - x_r)**2 + (y_l - y_r)**2)

FK = T(thetaL_sym, w/2, 0)@T((sp.pi - thetaL_sym + theta3_sym), l1, 0)@sp.Matrix([L/2, -sp.sqrt(l2**2 - (L/2)**2), 1])
FK = FK[:2,:]
FK_fast = lambdify((thetaR_sym, thetaL_sym), FK) # (x,y) = FK_fast(thetaR,thetaL)

J = FK.jacobian([thetaR_sym, thetaL_sym]).evalf()
J_fast = lambdify((thetaR_sym, thetaL_sym), J) # J_fast(thetaR,thetaL)

def IK_5_link(x, y, l1 = 0.09, l2 = 0.16, w = 0.07):

    def leg_wide(var):
        return np.linalg.norm([var[0], var[1] - np.pi]) 

    def x_constraint_equation(var):
    # should be equal to zero when the 
        return l1**2 - l2**2 + (x - w/2)**2 + y**2 - 2*l1*(y*np.sin(var[0]) + (x - w/2)*np.cos(var[0]))

    def y_constraint_equation(var):
        return l1**2 - l2**2 + (x + w/2)**2 + y**2 - 2*l1*(y*np.sin(var[1]) + (x + w/2)*np.cos(var[1]))

    res = minimize(leg_wide, (0.1, 9*np.pi/10), method="SLSQP", constraints= ({"type": "eq", "fun": x_constraint_equation}, 
                                                                              {"type": "eq", "fun": y_constraint_equation}))
    return (res.x[0], res.x[1])

## Connect to Odrive

In [6]:
def connect_odrive(serial):
    odrv0 = odrive.find_any(serial_number = serial, timeout = 20)
    if odrv0 is not None:
        print('Connected!')
        print('Target serial {} \t Odrive serial {}'.format(serial, odrv0.serial_number))
        print('Motor 0 calibrated: {}'.format(odrv0.axis0.motor.is_calibrated))
        print('Motor 1 calibrated: {}'.format(odrv0.axis1.motor.is_calibrated))    
    else:
        print("ODrive ({}) not found".format(serial))
    return odrv0

# odrv1 = connect_odrive("206A33A5304B")
# odrv2 = connect_odrive("335C315E3536")
odrv1 = connect_odrive("306539533235")
odrv2 = connect_odrive("20593884304E")

m0  = odrv1.axis0
m1  = odrv1.axis1
m00 = odrv2.axis0
m11 = odrv2.axis1

Connected!
Target serial 306539533235 	 Odrive serial 53211311583797
Motor 0 calibrated: False
Motor 1 calibrated: False
Connected!
Target serial 20593884304E 	 Odrive serial 35567572365390
Motor 0 calibrated: False
Motor 1 calibrated: False


## Setup

In [7]:
def SetParameter(odrv0):
    current_limit = 20
    odrv0.axis0.motor.config.current_lim    = current_limit #A
    odrv0.axis1.motor.config.current_lim    = current_limit #A
    odrv0.axis0.controller.config.vel_limit = 1000000
    odrv0.axis1.controller.config.vel_limit = 1000000
    odrv0.axis0.controller.config.pos_gain  = 50
    odrv0.axis1.controller.config.pos_gain  = 50
    odrv0.axis0.controller.config.vel_integrator_gain = 0
    odrv0.axis1.controller.config.vel_integrator_gain = 0
    return(current_limit)

def CheckMotorError(odrv0):
    axis0error = odrv0.axis0.error 
    axis1error = odrv0.axis1.error 
    axis0motorerror = odrv0.axis0.motor.error 
    axis1motorerror = odrv0.axis1.motor.error 
    axis0controllererror = odrv0.axis0.controller.error 
    axis1controllererror = odrv0.axis0.controller.error 
    print("error:          ", axis0error, axis1error)
    print("motorerror:     ", axis0motorerror, axis1motorerror)
    print("controllererror:", axis0controllererror, axis1controllererror)
    return()

def MotorClearError(odrv0):
    odrv0.axis0.error = 0
    odrv0.axis1.error = 0
    odrv0.axis0.motor.error = 0
    odrv0.axis1.motor.error = 0
    odrv0.axis0.controller.error = 0
    odrv0.axis1.controller.error = 0
    odrv0.axis0.encoder.error = 0
    odrv0.axis1.encoder.error = 0
    return()

In [8]:
%qtconsole

In [9]:
SetParameter(odrv1)
SetParameter(odrv2)
print("Parameter Set!")

Parameter Set!


In [10]:
print("odrive1")
CheckMotorError(odrv1)
print("odrive2")
CheckMotorError(odrv2)

MotorClearError(odrv1)
MotorClearError(odrv2)
print("Error Clear!")

odrive1
error:           0 0
motorerror:      0 0
controllererror: 0 0
odrive2
error:           0 0
motorerror:      0 0
controllererror: 0 0
Error Clear!


## Motor Commands

In [11]:
def Calibration(odrv0):
    odrv0.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    odrv0.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    print('Motor 0 calibrated: {}'.format(odrv0.axis0.motor.is_calibrated))
    print('Motor 1 calibrated: {}'.format(odrv0.axis1.motor.is_calibrated))   

def ClosedLoopControlMode(odrv0):
    odrv0.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
    odrv0.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

def IdleMode(odrv0):
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

def ControllerPositionControl(odrv0): # default mode
    odrv0.axis0.controller.config.control_mode = CTRL_MODE_POSITION_CONTROL
    odrv0.axis1.controller.config.control_mode = CTRL_MODE_POSITION_CONTROL
    print("Control Mode :{}".format(odrv0.axis0.controller.config.control_mode))

def ControllerCurrentControl(odrv0):
    odrv0.axis0.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL
    odrv0.axis1.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL
    print("Control Mode :{}".format(odrv0.axis0.controller.config.control_mode))

def MaxCurrentProtection(current, current_limit = 20):
    current = min(np.abs(current), current_limit) * np.sign(current) 
    return(current)

In [12]:
Calibration(odrv1)
Calibration(odrv2)

Motor 0 calibrated: False
Motor 1 calibrated: False
Motor 0 calibrated: False
Motor 1 calibrated: False


In [25]:
ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)

In [ ]:
ControllerPositionControl(odrv1)
ControllerPositionControl(odrv2)

In [24]:
ControllerCurrentControl(odrv1)
ControllerCurrentControl(odrv2)

Control Mode :1
Control Mode :1


In [15]:
IdleMode(odrv1)
IdleMode(odrv2)

# Define zero angle (Joint space)

In [16]:
#odrv1
m0_home_pos = m0.encoder.pos_estimate
m1_home_pos = m1.encoder.pos_estimate
theta_R_home = -((m0_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_home = -((m1_home_pos) / 8192 * 2 * np.pi)
print(m0_home_pos)
print(m1_home_pos)

-18.00002670288086
0.23437637090682983


In [17]:
#odrv2
m00_home_pos = m00.encoder.pos_estimate
m11_home_pos = m11.encoder.pos_estimate
theta_R_home = -((m00_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_home = -((m11_home_pos) / 8192 * 2 * np.pi)
print(m00_home_pos)
print(m11_home_pos)

579.75
-631.015625


# Define tiptoe home location (Joint space)

In [18]:
#odrv1
m0_equilibrium_toe_pos = m0.encoder.pos_estimate
m1_equilibrium_toe_pos = m1.encoder.pos_estimate
theta_R_eq = - ((m0_equilibrium_toe_pos - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_eq = - ((m1_equilibrium_toe_pos - m1_home_pos) / 8192 * 2 * np.pi)
theta_R_eq_d = theta_R_eq / (2 * np.pi) * 360
theta_L_eq_d = theta_L_eq / (2 * np.pi) * 360
print("Odrv1 \nR:", theta_R_eq_d, "\nL:", theta_L_eq_d)

Odrv1 
R: 180.0 
L: -0.0


In [19]:
#odrv2
m00_equilibrium_toe_pos = m00.encoder.pos_estimate
m11_equilibrium_toe_pos = m11.encoder.pos_estimate
theta_R_eq = - ((m00_equilibrium_toe_pos - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_eq = - ((m11_equilibrium_toe_pos - m11_home_pos) / 8192 * 2 * np.pi)
theta_R_eq_d = theta_R_eq / (2 * np.pi) * 360
theta_L_eq_d = theta_L_eq / (2 * np.pi) * 360
print("Odrv2 \nR:", theta_R_eq_d, "\nL:", theta_L_eq_d)

Odrv2 
R: 180.0 
L: -0.0


# Sine Trajectory (Foot Space)

In [ ]:
# DownAMP = 0.01
# UpperAMP = 0.05
# StanceHeight = 0.18
# StepLength = 0.12

# CurrentPercent = np.array([a/100+0.01 for a in range(100)]) # 0.01 ~ 1.00
# StancePercent = 0.6
# SwingPercent = 1 - StancePercent

# x = np.zeros(len(CurrentPercent), dtype=float)
# y = np.zeros(len(CurrentPercent), dtype=float)

# for i in range(len(CurrentPercent)):
#      if (CurrentPercent[i] <= StancePercent):
#           x[i] = -(StepLength / 2) + (CurrentPercent[i] / StancePercent) * StepLength
#           y[i] =  DownAMP * np.sin(np.pi * CurrentPercent[i] / StancePercent) + StanceHeight
#      else:
#           x[i] = (StepLength / 2) - ((CurrentPercent[i] - StancePercent) / SwingPercent) * StepLength
#           y[i] = -UpperAMP * np.sin(np.pi * (CurrentPercent[i] - StancePercent) / SwingPercent) + StanceHeight

# plt.plot(x,y)
# plt.plot(x[99],y[99],x[79],y[79],x[59],y[59],x[29],y[29], marker='*')
# plt.title('SineTrajectory')
# plt.ylim(0.195, 0.125)
# plt.xlim(0.065, -0.065)
# plt.grid()
# plt.show()

In [20]:
"""
default offset = 0.02m for imporving performance
"""
def RightTrajectory(DownAMP = 0.01, UpperAMP = 0.07, StanceHeight = 0.185, StepLength = 0.18, offset = 0.02):
    
    StancePercent = 0.5
    SwingPercent = 1 - StancePercent
    t_iteration = 4
    t_real = time.time()
    t_cycle = t_real % 4
    CurrentPercent = t_cycle / t_iteration
    
    # stance phase
    if (CurrentPercent <= StancePercent):
        x = -(StepLength / 2) + (CurrentPercent / StancePercent) * StepLength + offset
        y =  DownAMP * np.sin(np.pi * CurrentPercent / StancePercent) + StanceHeight
        
    # swing phase    
    else: 
        x = (StepLength / 2) - ((CurrentPercent - StancePercent) / SwingPercent) * StepLength + offset
        y = -UpperAMP * np.sin(np.pi * (CurrentPercent - StancePercent) / SwingPercent) + StanceHeight

    return np.array([[x], [y]])

def LeftTrajectory(DownAMP = 0.01, UpperAMP = 0.07, StanceHeight = 0.185, StepLength = 0.18, offset = 0.02):
    
    StancePercent = 0.5
    SwingPercent = 1 - StancePercent
    t_iteration = 4
    t_real = time.time()
    t_cycle = t_real % 4
    CurrentPercent = t_cycle / t_iteration
    
    # swing phase
    if (CurrentPercent <= StancePercent): 
        x = (StepLength / 2)- (CurrentPercent / StancePercent) * StepLength + offset
        y = -UpperAMP * np.sin(np.pi * (CurrentPercent / StancePercent)) + StanceHeight
        
    # stance phase                           
    else: 
        x = -(StepLength / 2) + ((CurrentPercent - StancePercent) / SwingPercent) * StepLength + offset
        y =  DownAMP * np.sin(np.pi * ((CurrentPercent - StancePercent) / SwingPercent)) + StanceHeight
        
    return np.array([[-x], [y]])

# Position control mode

In [21]:
ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)

t_start = time.time()
duration = 10 #sec
count = 0

while (time.time() - t_start < duration):
    
    t0 = time.time()
    
    odrv1_theta_R = IK_5_link(RightTrajectory()[0],RightTrajectory()[1])[1]
    odrv1_theta_L = IK_5_link(RightTrajectory()[0],RightTrajectory()[1])[0]
    odrv2_theta_R = IK_5_link(LeftTrajectory()[0],LeftTrajectory()[1])[1]
    odrv2_theta_L = IK_5_link(LeftTrajectory()[0],LeftTrajectory()[1])[0]
    
    #full function of both legs
    
    m0_pos  = -(odrv1_theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
    m1_pos  = -odrv1_theta_L/(2*np.pi)*8192 + m1_home_pos
    m00_pos = -(odrv2_theta_R - np.pi)/(2*np.pi)*8192 + m00_home_pos
    m11_pos = -odrv2_theta_L/(2*np.pi)*8192 + m11_home_pos
    
    m0.controller.set_pos_setpoint(m0_pos,0,0)
    m1.controller.set_pos_setpoint(m1_pos,0,0)
    m00.controller.set_pos_setpoint(m00_pos,0,0)
    m11.controller.set_pos_setpoint(m11_pos,0,0)
   
    count += 1

    t1 = time.time()
    time_spent = t1 - t0

print(count,'iterations in', duration, 'second')
print("Frequency:", count/duration, 'Hz')

756 iterations in 10 second
Frequency: 75.6 Hz


In [ ]:
ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)

t_start = time.time()
duration = 10 #sec
count = 0

while (time.time() - t_start < duration):
    
    t0 = time.time()
    
    odrv2_theta_R = IK_5_link(RightTrajectory()[0],RightTrajectory()[1])[1]
    odrv2_theta_L = IK_5_link(RightTrajectory()[0],RightTrajectory()[1])[0]
    odrv1_theta_R = IK_5_link(LeftTrajectory()[0],LeftTrajectory()[1])[1]
    odrv1_theta_L = IK_5_link(LeftTrajectory()[0],LeftTrajectory()[1])[0]
    
    #full function of both legs
    
    m0_pos  = -(odrv1_theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
    m1_pos  = -odrv1_theta_L/(2*np.pi)*8192 + m1_home_pos
    m00_pos = -(odrv2_theta_R - np.pi)/(2*np.pi)*8192 + m00_home_pos
    m11_pos = -odrv2_theta_L/(2*np.pi)*8192 +m11_home_pos
    
    m0.controller.set_pos_setpoint(m0_pos,0,0)
    m1.controller.set_pos_setpoint(m1_pos,0,0)
    m00.controller.set_pos_setpoint(m00_pos,0,0)
    m11.controller.set_pos_setpoint(m11_pos,0,0)
   
    count += 1

    t1 = time.time()
    time_spent = t1 - t0

print(count,'iterations in', duration, 'second')
print("Frequency:", count/duration, 'Hz')

In [22]:
IdleMode(odrv1)
IdleMode(odrv2)

# Define virtual compliance

In [23]:
'''
    k  = np.array([[kx,0],[0,ky]])
    c  = np.array([[cx,0],[0,cy]])
'''
def cartesian_compliance_homePosition(theta_R, theta_L, vel_R, vel_L):
    k  = np.array([[110,0],[0,110]])
    c  = np.array([[0.5,0],[0,0.5]])
    Kt = 0.0285
    J_trans  = -np.transpose(J_fast(theta_R, theta_L))  
    disp     = FK_fast(theta_R, theta_L) - FK_fast(theta_R_eq, theta_L_eq) 
    vel      = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    currents = J_trans @ ( k @ disp + c @ vel )*(1/Kt)
    return currents

def cartesian_compliance_RightTrajectory(theta_R, theta_L, vel_R, vel_L):
    J_trans = -np.transpose(J_fast(theta_R, theta_L))
    disp    = FK_fast(theta_R, theta_L) - RightTrajectory()   
    vel     = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    Kt = 0.0285
    if RightTrajectory()[1] < 0.185:
        k = np.array([[110,0],[0,110]])
        c = np.array([[0.5,0],[0,0.5]])
        a = 1
    else:
        k = np.array([[110,0],[0,110]])
        c = np.array([[0.5,0],[0,0.5]])
        a = 0
    currents = J_trans @ ( k @ disp + c @ vel )*(1/Kt)
    return np.array([currents[0], currents[1], a])

def cartesian_compliance_LeftTrajectory(theta_R, theta_L, vel_R, vel_L):
    J_trans = -np.transpose(J_fast(theta_R, theta_L))
    disp    = FK_fast(theta_R, theta_L) - LeftTrajectory()
    vel     = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    Kt = 0.0285
    if LeftTrajectory()[1] < 0.185:
        k = np.array([[110,0],[0,110]])
        c = np.array([[0.5,0],[0,0.5]])
        a = 1
    else:
        k = np.array([[110,0],[0,110]])
        c = np.array([[0.5,0],[0,0.5]])
        a = 0
    currents = J_trans @ ( k @ disp + c @ vel )*(1/Kt)
    return np.array([currents[0], currents[1], a])


# Current Control loop (each loop takes about 0.005s 200Hz)

In [26]:
ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)

odrv1_cur_l = []
odrv1_cur_r = []
odrv2_cur_l = []
odrv2_cur_r = []

t_start = time.time()
duration = 10 #sec
count = 0

while (time.time() - t_start < duration):
    
    t0 = time.time()
    
    #current angle(rad) & velocity(rad/s)
    odrv1_theta_R = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
    odrv1_theta_L = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
    odrv2_theta_R = - ((m00.encoder.pos_estimate - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
    odrv2_theta_L = - ((m11.encoder.pos_estimate - m11_home_pos) / 8192 * 2 * np.pi)
    
    odrv1_vel_R = - ((m0.encoder.vel_estimate) / 8192 * 2 * np.pi)
    odrv1_vel_L = - ((m1.encoder.vel_estimate) / 8192 * 2 * np.pi)
    odrv2_vel_R = - ((m00.encoder.vel_estimate) / 8192 * 2 * np.pi)
    odrv2_vel_L = - ((m11.encoder.vel_estimate) / 8192 * 2 * np.pi)
    
    #get current
    joint_damping_R = 0.5
    joint_damping_L = 0.5
    
    # run the commands based on time
    odrv1_current = cartesian_compliance_RightTrajectory(odrv1_theta_R, odrv1_theta_L, odrv1_vel_R, odrv1_vel_L)[0:2]
    phase_indicator1 = cartesian_compliance_RightTrajectory(odrv1_theta_R, odrv1_theta_L, odrv1_vel_R, odrv1_vel_L)[2]
#     odrv1_current = cartesian_compliance_homePosition(odrv1_theta_R, odrv1_theta_L, odrv1_vel_R, odrv1_vel_L)
    odrv1_damping_current_R = np.array(joint_damping_R * odrv1_vel_R)
    odrv1_damping_current_L = np.array(joint_damping_L * odrv1_vel_L)
    odrv1_current_R = odrv1_current[0] - odrv1_damping_current_R
    odrv1_current_L = odrv1_current[1] - odrv1_damping_current_L
    
    odrv2_current = cartesian_compliance_LeftTrajectory(odrv2_theta_R, odrv2_theta_L, odrv2_vel_R, odrv2_vel_L)[0:2]
    phase_indicator2 = cartesian_compliance_LeftTrajectory(odrv2_theta_R, odrv2_theta_L, odrv2_vel_R, odrv2_vel_L)[2]
#     odrv2_current = cartesian_compliance_homePosition(odrv2_theta_R, odrv2_theta_L, odrv2_vel_R, odrv2_vel_L)
    odrv2_damping_current_R = np.array(joint_damping_R * odrv2_vel_R)
    odrv2_damping_current_L = np.array(joint_damping_L * odrv2_vel_L)
    odrv2_current_R = odrv2_current[0] - odrv2_damping_current_R
    odrv2_current_L = odrv2_current[1] - odrv2_damping_current_L

    #make sure the current is not exceeds the limit
    odrv1_current_R = MaxCurrentProtection(odrv1_current_R)
    odrv1_current_L = MaxCurrentProtection(odrv1_current_L)
    odrv2_current_R = MaxCurrentProtection(odrv2_current_R)
    odrv2_current_L = MaxCurrentProtection(odrv2_current_L)
    
    if phase_indicator1 ==1:
        m0.controller.set_current_setpoint(-odrv1_current_R)
        m1.controller.set_current_setpoint(0) #rear motor
        m00.controller.set_current_setpoint(-odrv2_current_R)
        m11.controller.set_current_setpoint(-odrv2_current_L)
    else:
        m0.controller.set_current_setpoint(-odrv1_current_R)
        m1.controller.set_current_setpoint(-odrv1_current_L)
        m00.controller.set_current_setpoint(0) #rear motor
        m11.controller.set_current_setpoint(-odrv2_current_L)
        
#     #control the motors
#     m0.controller.set_current_setpoint(-odrv1_current_R)
#     m1.controller.set_current_setpoint(-odrv1_current_L)
#     m00.controller.set_current_setpoint(-odrv2_current_R)
#     m11.controller.set_current_setpoint(-odrv2_current_L)
    
    #plotting
    odrv1_cur_r.append(-odrv1_current_R)    
    odrv1_cur_l.append(-odrv1_current_L) 
    odrv2_cur_r.append(-odrv2_current_R)    
    odrv2_cur_l.append(-odrv2_current_L) 
    
    count += 1

    t1 = time.time()
    time_spent = t1 - t0

print(count,'iterations in', duration, 'second')
print("Frequency:", count/duration, 'Hz')

m0.controller.set_current_setpoint(0)
m1.controller.set_current_setpoint(0)
m00.controller.set_current_setpoint(0)
m11.controller.set_current_setpoint(0)

ControllerPositionControl(odrv1)
ControllerPositionControl(odrv2)

ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)
    

KeyboardInterrupt: 

In [ ]:
# plot the current
i = np.arange(count)

plt.subplot(1, 2, 1)
plt.plot(i,odrv1_cur_r,'tab:orange',linewidth=1)
plt.plot(i,odrv1_cur_l,'tab:green',linewidth=1)
plt.legend(['front-m0','back-m1'],loc='upper left',ncol=2, mode="expand")
plt.title('Odrv1 (R_leg)')
plt.ylim(-20,20)
plt.grid(True)

plt.ylabel('Current (A)')

plt.subplot(1, 2, 2)
plt.plot(i,odrv2_cur_r,'tab:red',linewidth=1)
plt.plot(i,odrv2_cur_l,'tab:blue',linewidth=1)
plt.legend(['back-m0','front-m1'],loc='upper left',ncol=2, mode="expand")
plt.title('Odrv2 (L_leg)')
plt.ylim(-20,20)
plt.grid(True)

plt.suptitle('Gait Period: 2s / Total elapse: {}s'.format(duration))

plt.show()
plt.pause(0.01)

In [ ]:
%matplotlib qt

In [ ]:
IdleMode(odrv1)
IdleMode(odrv2)

In [ ]:
ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)

In [ ]:
print("odrive1")
CheckMotorError(odrv1)
print("odrive2")
CheckMotorError(odrv2)

MotorClearError(odrv1)
MotorClearError(odrv2)
print("Error Clear!")